In [1]:
# Issues with hyperdrive approach
# Minimal feedback in logs (so almost impossible to debug if something doesn't work during runtime)
# argparse.ArgumentParser() as arguments that could be passed? Why not just create a function instead?
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227857
Azure region: westeurope
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-227857


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-project"

### YOUR CODE HERE ###

## Used some code from train-hyperparameter-tune-deploy-with-keras
# choose a name for your cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-03-14T22:55:44.058000+00:00', 'errors': None, 'creationTime': '2023-03-14T22:55:39.507454+00:00', 'modifiedTime': '2023-03-14T22:55:45.979782+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive import choice, loguniform
import os
from azureml.core import Environment

# Specify parameter sampler
# Using random sampling for faster hyperparameter tuning as covered earlier in the course
# And a 2 layer neural network
# PARAMETERS OF THE MODEL WE SPECIFIED IN train.py LOGISTIC REGRESSION (NOT A NEURAL NETWORK SO HARD TO MODIFY THIS)
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5, 1, 2),
        '--max_iter': choice(100, 200, 500, 1000),
    }
)

# ps = RandomParameterSampling(
#     {
#         '--batch-size': choice(25, 50, 100),
#         '--first-layer-neurons': choice(10, 25, 50, 100),
#         '--second-layer-neurons': choice(10, 25, 50, 100),
#         '--learning-rate': loguniform(-6, -1)
#     }
# )


# Specify a Policy
# Using bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
# keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

# parameters we'd use in the train.py (if some of them aren't hyperparameters to be tuned could specify them here, in this case keeping it empty)
# Once I find the optimal hyperparameters can specify them as args (i.e. the exmaple below)
# args = ['--C', 1.0,
#         '--max_iter', 100]

args = []

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = './',
                      script='train.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [4]:

# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
# obtained from Azure's train-hyperparameter-tune-deploy-with-keras notebook
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show() # Showing run details with the widget
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_736d01b4-9cd4-48a5-8f39-df84e75b774a
Web View: https://ml.azure.com/runs/HD_736d01b4-9cd4-48a5-8f39-df84e75b774a?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-227857/workspaces/quick-starts-ws-227857&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-14T22:55:56.418330][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-14T22:55:56.9613928Z][SCHEDULER][INFO]Scheduling job, id='HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_0' 
[2023-03-14T22:55:57.0720223Z][SCHEDULER][INFO]Scheduling job, id='HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_1' 
[2023-03-14T22:55:57.2121931Z][SCHEDULER][INFO]Scheduling job, id='HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_2' 
[2023-03-14T22:55:57.3165077Z][SCHEDULER][INFO]Scheduling job, id='HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_3' 
[2023-03-14T22:55:57.293483][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_736d01b4-9cd4-48a5-8f39-df84e75b774a',
 'target': 'compute-project',
 'status': 'Completed',
 'startTimeUtc': '2023-03-14T22:55:55.50659Z',
 'endTimeUtc': '2023-03-14T23:17:29.901023Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '22da472e-f8c8-4b2a-99fc-45a6253aa085',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '16',
  'score': '0.9132018209408195',
  'best_child_run_id': 'HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_8',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_736d01b4-9cd4-48a5-8f39-df84e75b774a_8'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

In [21]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
os.makedirs('./outputs', exist_ok=True)



# Saving best model - had several issues with bestrun.register model with it thinking the output directory didn't exist so saving model right in main directory
model_hyperdrive = best_run.register_model(model_name='hyperdrive_model', model_path = './')

['--C', '2', '--max_iter', '1000']
['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-03-14-23', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
# Using azure dataset
ds = TabularDatasetFactory().from_delimited_files(path=["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [22]:
from train import clean_data

# Use the clean_data function from the train.py file to clean your data.
x, y = clean_data(ds)

# All data must be merged into a single dataframe for Automlconfig
df_all = x.merge(y,left_index=True,right_index=True)

In [38]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=df_all,
    label_column_name='y',
    n_cross_validations=5)

In [39]:
# Submit your automl run
exp_automl = Experiment(workspace=ws, name="udacity-project")

# Submit experiment to experiment object created at start of notebook (also used for hyperdrive)
automl_run = exp_automl.submit(config=automl_config, show_output= True)

2023-03-15:00:06:02,508 WARNING  [local_experiment_driver.py:147] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_328e4be0-baa8-49bd-b662-76f39ccef419,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/03/15 00:15:05 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-03-15:00:40:36,596 INFO     [explanation_client.py:334] Using default datastore for uploads


In [42]:
# Retrieve best model 

# Get model
best_run = automl_run.get_best_child()

# Saving best model
model_automl = best_run.register_model(model_name='automl_model', model_path = './')